In [10]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd



In [11]:
df = pd.read_csv('creditcard.csv')
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.165980e-15,3.416908e-16,-1.373150e-15,2.086869e-15,9.604066e-16,1.490107e-15,-5.556467e-16,1.177556e-16,-2.406455e-15,...,1.656562e-16,-3.444850e-16,2.578648e-16,4.471968e-15,5.340915e-16,1.687098e-15,-3.666453e-16,-1.220404e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [12]:
fraud = df[df.Class==1].index
no_of_frauds = len(fraud)

normal = df[df.Class==0].index
no_of_normal = len(normal)

print("Normal Cases {}".format(no_of_normal))
print("Fraud Cases {}".format(no_of_frauds))
print("Ratio Of Fraud {}".format(no_of_frauds/(no_of_frauds+no_of_normal)))

Normal Cases 284315
Fraud Cases 492
Ratio Of Fraud 0.001727485630620034


# Auto Encoders

In [13]:
train_set = df.sample(frac=0.75,replace=False,random_state=123)
test_set = df.loc[ set(df.index) - set(train_set.index)]

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df.drop(['Class','Time'],axis=1))
scaled_test_set = scaler.transform(test_set.drop(['Class','Time'],axis=1))
scaled_train_set = scaler.transform(train_set.drop(['Class','Time'],axis=1))
print("Length Of Train {}".format(len(scaled_train_set)))
print("Length Of Test {}".format(len(scaled_test_set)))

Length Of Train 213605
Length Of Test 71202


In [15]:
num_input = len(scaled_train_set[1])
num_hidden = 2
num_output = num_input

learning_rate = 0.01
keep_prob = 0.5
tf.compat.v1.reset_default_graph()

In [16]:
tf.compat.v1.disable_eager_execution()
x = tf.compat.v1.placeholder(tf.float32,shape = [None,num_input])

#weight

initializer = tf.compat.v1.variance_scaling_initializer()
w = tf.Variable(initializer([num_input,num_hidden]))
w_out = tf.Variable(initializer([num_hidden,num_output]))

b = tf.Variable(tf.zeros(num_hidden))
b_out = tf.Variable(tf.zeros(num_output))

#activation
act_func = tf.nn.tanh
#layer

hidden_layer = act_func(tf.matmul(x,w)+b)
dropout_layer = tf.nn.dropout(hidden_layer,rate=1-keep_prob)
output_layer = tf.matmul(dropout_layer,w_out) + b_out 

In [17]:
import numpy as np
loss = tf.reduce_mean(tf.abs(output_layer-x))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)
init = tf.compat.v1.global_variables_initializer()

def next_batch(x_data,batch_size):
    rindex = np.random.choice(x_data.shape[0],batch_size,replace=False)
    x_batch = x_data[rindex,:]
    return x_batch

# Training

In [18]:
num_step = 10
batch_size = 150
num_batches = len(scaled_train_set)//batch_size
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for step in range(num_step):
        for iteration in range(num_batches):
            x_batch = next_batch(scaled_train_set,batch_size)
            sess.run(train,feed_dict={x:scaled_train_set})
        if step % 1 == 0:
            err = loss.eval(feed_dict={x:scaled_train_set})
            print(step, "\tLoss : ",err)
            output_2d = hidden_layer.eval(feed_dict={x:scaled_train_set})
    output_2d = hidden_layer.eval(feed_dict={x:scaled_train_set})

KeyboardInterrupt: 

In [ ]:
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.figure(figsize=(20,8))
plt.scatter(output_2d[:,0],output_2d[:,1],c=train_set['Class'],alpha=0.7)

# Linear Classification


In [ ]:
y = df.Class
x = df.drop(['Class','Time'],axis=1)
x_scaled = (x-x.min())/(x.max()-x.min()) #feature scaling 0-1

In [ ]:
nv1 = tf.feature_column.numeric_column('V1')
nv2 = tf.feature_column.numeric_column('V2')
nv3 = tf.feature_column.numeric_column('V3')
nv4 = tf.feature_column.numeric_column('V4')
nv5 = tf.feature_column.numeric_column('V5')
nv6 = tf.feature_column.numeric_column('V6')
nv7 = tf.feature_column.numeric_column('V7')
nv8 = tf.feature_column.numeric_column('V8')
nv9 = tf.feature_column.numeric_column('V9')
nv10 = tf.feature_column.numeric_column('V10')
nv11 = tf.feature_column.numeric_column('V11')
nv12 = tf.feature_column.numeric_column('V12')
nv13 = tf.feature_column.numeric_column('V13')
nv14 = tf.feature_column.numeric_column('V14')
nv15 = tf.feature_column.numeric_column('V15')
nv16 = tf.feature_column.numeric_column('V16')
nv17 = tf.feature_column.numeric_column('V17')
nv18 = tf.feature_column.numeric_column('V18')
nv19 = tf.feature_column.numeric_column('V19')
nv20 = tf.feature_column.numeric_column('V20')
nv21 = tf.feature_column.numeric_column('V21')
nv22 = tf.feature_column.numeric_column('V22')
nv23 = tf.feature_column.numeric_column('V23')
nv24 = tf.feature_column.numeric_column('V24')
nv25 = tf.feature_column.numeric_column('V25')
nv26 = tf.feature_column.numeric_column('V26')
nv27 = tf.feature_column.numeric_column('V27')
nv28 = tf.feature_column.numeric_column('V28')
nv30 = tf.feature_column.numeric_column('Amount')


In [ ]:
features = [nv1,nv2,nv3,nv4,nv5,nv6,nv7,nv8,nv9,nv10,nv11,nv12,nv13,nv14,nv15,nv16,nv17,nv18,nv19,nv20,nv21,nv22,nv23,nv24,nv25,nv26,nv27,nv28,nv30,]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x_scaled,y,train_size=0.75,random_state=101)

In [ ]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=1000,shuffle=True)
model = tf.compat.v1.estimator.LinearClassifier(feature_columns=features,n_classes=2)
model.train(input_func,steps=1000)
#test on training data
result = model.evaluate(tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1,shuffle=False))

In [ ]:
print(result)

In [ ]:
test_eval = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)
result = model.evaluate(test_eval)


In [ ]:
print("***************Results****************")
print(result)

In [ ]:
result = model.predict(X_test.iloc[1])
print(y_test.iloc[1])